**machine translation**

In this example the ``OpenNMT-py`` library is used to demnstrate a **N**eural **M**achine **T**ranslation task.
The code in this sample follows the instruction on the ``quickstart`` example found on [OpenNMT-py:Quickstart](https://github.com/OpenNMT/OpenNMT-py#quickstart)

[installation](#install)<br>
[download data](#download)<br>
[prepare data](#prepare)<br>
[train model](#train)<br>
[translate](#translate)<br>

<a id=#install>**Installation**</a>

In [1]:
# Install OpenNMT-py 2.x
# NOTE: By the end of the insatallation, it might ask for restarting the runtime...
# In this case, just click the "RESTART RUNTIME" button.

!pip3 install git+https://github.com/OpenNMT/OpenNMT-py.git

  Cloning https://github.com/OpenNMT/OpenNMT-py.git to /tmp/pip-req-build-mig8avoq
  Running command git clone -q https://github.com/OpenNMT/OpenNMT-py.git /tmp/pip-req-build-mig8avoq
     |████████████████████████████████| 81kB 6.3MB/s 
     |████████████████████████████████| 748.8MB 25kB/s 
     |████████████████████████████████| 81kB 12.4MB/s 
     |████████████████████████████████| 51kB 9.1MB/s 
     |████████████████████████████████| 61kB 9.9MB/s 
     |████████████████████████████████| 276kB 54.9MB/s 
     |████████████████████████████████| 2.5MB 54.4MB/s 
     |████████████████████████████████| 1.2MB 37.8MB/s 
  Created wheel for OpenNMT-py: filename=OpenNMT_py-2.0.1-cp36-none-any.whl size=208258 sha256=7b38de968cbb3ad6f8e2408a1400a4a73c2300d53385d5dfdaed07375de808eb
  Stored in directory: /tmp/pip-ephem-wheel-cache-hi40jjuc/wheels/ba/73/26/44c735cd015b6622c329f69e7ee77a982d78390e87da0fa6f8
  Created wheel for configargparse: filename=ConfigArgParse-1.2.3-cp36-none-any.whl size=

In [2]:
# On Google Colab ONLY
# Reinstall Torch to avoid incompatibility with Cuda 10.1

# NOTE: By the end of the insatallation, it might ask for restarting the runtime...
# In this case, just click the "RESTART RUNTIME" button.

!pip3 install --ignore-installed torch==1.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 708.0MB 17kB/s 
     |████████████████████████████████| 14.8MB 218kB/s 
     |████████████████████████████████| 829kB 49.3MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=cbeacf1ba8b2612b8217b02bfd191e43ffaa3cb58452931b8a8fe9e6e768fdfc
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.6.0+cu101 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


<a id="download"> **download files** </a>

In [1]:
# Download the files of the QuickStart

!wget https://s3.amazonaws.com/opennmt-trainingdata/toy-ende.tar.gz
!tar xf toy-ende.tar.gz

--2021-02-02 21:31:19--  https://s3.amazonaws.com/opennmt-trainingdata/toy-ende.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.94.6
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.94.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1662081 (1.6M) [application/x-gzip]
Saving to: ‘toy-ende.tar.gz.1’

toy-ende.tar.gz.1   100%[===================>]   1.58M  4.08MB/s    in 0.4s    

2021-02-02 21:31:20 (4.08 MB/s) - ‘toy-ende.tar.gz.1’ saved [1662081/1662081]



In [2]:
# Optional: List the extracted files

!cd toy-ende/ && ls

run	      src-train.txt  tgt-test.txt   tgt-val.txt
src-test.txt  src-val.txt    tgt-train.txt


In [3]:
# Optional: Print the first 3 lines of the source file

!head -n 3 toy-ende/src-train.txt

It is not acceptable that , with the help of the national bureaucracies , Parliament &apos;s legislative prerogative should be made null and void by means of implementing provisions whose content , purpose and extent are not laid down in advance .
Federal Master Trainer and Senior Instructor of the Italian Federation of Aerobic Fitness , Group Fitness , Postural Gym , Stretching and Pilates; from 2004 , he has been collaborating with Antiche Terme as personal Trainer and Instructor of Stretching , Pilates and Postural Gym .
&quot; Two soldiers came up to me and told me that if I refuse to sleep with them , they will kill me . They beat me and ripped my clothes .


In [4]:
# Optional: Check the number of lines in the source file

!echo "Number of lines:" && wc -l toy-ende/src-train.txt

Number of lines:
10000 toy-ende/src-train.txt


<a id="prepare">**prepare data**</a>

In [5]:
# Create the YAML configuration file
# On a regular machine, you can create it manually or with nano

config = '''# toy_en_de.yaml

## Where the samples will be written
save_data: toy-ende/run/example

## Where the vocab(s) will be written
src_vocab: toy-ende/run/example.vocab.src
tgt_vocab: toy-ende/run/example.vocab.tgt

## Where the model will be saved
save_model: model/model

# Prevent overwriting existing files in the folder
overwrite: False

# Corpus opts:
data:
    corpus_1:
        path_src: toy-ende/src-train.txt
        path_tgt: toy-ende/tgt-train.txt
    valid:
        path_src: toy-ende/src-val.txt
        path_tgt: toy-ende/tgt-val.txt

world_size: 1
gpu_ranks: [0]

# Remove or modify these lines for bigger files
train_steps: 1000
valid_steps: 200
'''

with open("toy_en_de.yaml", "w+") as config_yaml:
  config_yaml.write(config)

!cat toy_en_de.yaml

# toy_en_de.yaml

## Where the samples will be written
save_data: toy-ende/run/example

## Where the vocab(s) will be written
src_vocab: toy-ende/run/example.vocab.src
tgt_vocab: toy-ende/run/example.vocab.tgt

## Where the model will be saved
save_model: model/model

# Prevent overwriting existing files in the folder
overwrite: False

# Corpus opts:
data:
    corpus_1:
        path_src: toy-ende/src-train.txt
        path_tgt: toy-ende/tgt-train.txt
    valid:
        path_src: toy-ende/src-val.txt
        path_tgt: toy-ende/tgt-val.txt

world_size: 1
gpu_ranks: [0]

# Remove or modify these lines for bigger files
train_steps: 1000
valid_steps: 200


In [6]:
# Build Vocabulary

!onmt_build_vocab -config toy_en_de.yaml -n_sample -1

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-02-02 21:31:21,270 INFO] Counter vocab from -1 samples.
[2021-02-02 21:31:21,270 INFO] n_sample=-1: Build vocab on full datasets.
[2021-02-02 21:31:21,278 INFO] corpus_1's transforms: TransformPipe()
[2021-02-02 21:31:21,278 INFO] Loading ParallelCorpus(toy-ende/src-train.txt, toy-ende/tgt-train.txt, align=None)...
[2021-02-02 21:31:21,580 INFO] Counters src:24995
[2021-02-02 21:31:21,581 INFO] Counters tgt:35816
Traceback (most recent call last):
  File "/usr/local/bin/onmt_build_vocab", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.6/dist-packages/onmt/bin/build_vocab.py", line 66, in main
    build_vocab_main(opts)
  File "/usr/local/lib/python3.6/dist-packages/onmt/bin/build_vocab.py", line 53, in build_vocab_main
    save_counter(src_counter, opts.src_vocab)
  File "/usr/local/lib/python3.6/dist-packages/onmt/bin/build_vocab.py", line 42, in save_counter
    check_path(save_path, 

In [7]:
# Check if GPU is active
# If not, go to "Runtime" menu > "Change runtime type" > "GPU"

!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-16e938e0-1c7b-2ec0-09ff-cc2851fb4d68)


In [8]:
# Make sure the GPU is visable to PyTorch

import torch

gpu_id = torch.cuda.current_device()
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(gpu_id))

True
Tesla T4


<a id="train">**train model** </a>

In [9]:
# Train the NMT model

!onmt_train -config toy_en_de.yaml

[2021-02-02 21:31:22,729 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2021-02-02 21:31:22,729 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-02-02 21:31:22,729 INFO] Missing transforms field for valid data, set to default: [].
[2021-02-02 21:31:22,729 INFO] Parsed 2 corpora from -data.
[2021-02-02 21:31:22,729 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2021-02-02 21:31:22,729 INFO] Loading vocab from text file...
[2021-02-02 21:31:22,729 INFO] Loading src vocabulary from toy-ende/run/example.vocab.src
[2021-02-02 21:31:22,775 INFO] Loaded src vocab has 24995 tokens.
[2021-02-02 21:31:22,785 INFO] Loading tgt vocabulary from toy-ende/run/example.vocab.tgt
[2021-02-02 21:31:22,851 INFO] Loaded tgt vocab has 35816 tokens.
[2021-02-02 21:31:22,864 INFO] Building fields with vocab in counters...
[2021-02-02 21:31:22,929 INFO]  * tgt vocab size: 35820.
[2021-02-02 21:31:22,960 INFO]  * src vocab siz

<a id="translate">**translate** </a>

In [10]:
# Translate

!onmt_translate -model model/model_step_1000.pt -src toy-ende/src-test.txt -output toy-ende/pred_1000.txt -gpu 0 -verbose

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
SENT 1738: ['Pope', 'Francis', 'to', 'name', 'first', 'cardinals', 'in', 'February']
PRED 1738: Das Hotel ist in der Nähe des
PRED SCORE: -21.9408

[2021-02-02 21:35:36,578 INFO] 
SENT 1739: ['Pope', 'Francis', 'will', 'create', 'new', 'cardinals', 'of', 'the', 'Catholic', 'Church', 'for', 'his', 'first', 'time', 'on', 'February', '22', ',', 'the', 'Vatican', 'announced', 'Thursday', '.']
PRED 1739: Die Kommission der Kommission in der Nähe der Europäischen Union in der Nähe der Europäischen Union in der Nähe des
PRED SCORE: -53.9640

[2021-02-02 21:35:36,578 INFO] 
SENT 1740: ['Cardinals', 'are', 'the', 'highest-ranking', 'clergy', 'in', 'the', 'Catholic', 'Church', 'below', 'the', 'pope', ',', 'and', 'they', '&apos;re', 'the', 'ones', 'who', 'elect', 'popes', ',', 'so', 'Francis', 'will', 'be', 'appointing', 'his', 'first', 'group', 'of', 'men', 'who', 'will', 'ultimately', 'help', 'choose', 'his', 'successor', '.']
P

Copyright © 2020 IUBH Internationale Hochschule